In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score, auc
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.model_selection import StratifiedKFold
import category_encoders
from catboost import Pool
from catboost.utils import get_roc_curve
from sklearn.metrics import auc


In [2]:
train_data = pd.read_csv('./train_dataset/train_public.csv')
test_public = pd.read_csv('./test_public.csv')
train_internet = pd.read_csv('./train_dataset/train_internet.csv', encoding='gb2312')

sub=pd.read_csv("./submit/test_newcb.csv")
sub=sub.rename(columns={'id': 'loan_id'})
sub.loc[sub['isDefault']<0.5,'isDefault'] = 0
nw_sub=sub[(sub['isDefault']==0)]
nw_test_data=test_public.merge(nw_sub,on='loan_id',how='inner')
nw_train_data = pd.concat([train_data,nw_test_data]).reset_index(drop=True)
nw_train_data.to_csv("./train_dataset/nw_train_public2.csv",index=0)

In [3]:
train_data = pd.read_csv('./train_dataset/nw_train_public2.csv')

In [4]:
X_train = train_data.copy()
X_test = test_public.copy()
X_train = X_train.drop(['isDefault'], axis = 1)
X_train.shape
X_train['is_train'] = 1
X_test['is_train'] = 0

X_traintest = X_traintest = pd.concat([X_train, X_test], axis = 0)
X_traintest.shape
X_internet = train_internet.copy()

### 数据探索 

In [5]:
unimport_fea = ['employer_type', 'industry', 'region', 'policy_code', 'pub_dero_bankrup', 'f1', 'del_in_18month', 'initial_list_status']
for cat in unimport_fea:
    X_internet = X_internet.drop(cat, axis=1)

In [6]:
unimport_fea = ['employer_type', 'industry', 'region',  'policy_code', 'pub_dero_bankrup', 'f1', 'del_in_18month', 'initial_list_status']
for cat in unimport_fea:
    X_traintest = X_traintest.drop(cat, axis=1)

### 特征工程 

#### work_year, class 

In [7]:
work_year_dict = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
}


X_traintest['work_year'] = X_traintest['work_year'].map(work_year_dict)
X_internet['work_year'] = X_internet['work_year'].map(work_year_dict)

In [8]:
class_dict = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
}


X_traintest['class'] = X_traintest['class'].map(class_dict)
X_internet['class'] = X_internet['class'].map(class_dict)

#### issue_date, earlies_credit_mon

In [9]:
X_traintest['issue_date'] = pd.to_datetime(X_traintest['issue_date'])
X_internet['issue_date'] = pd.to_datetime(X_internet['issue_date'])

X_traintest['issue_date_delta'] = (X_traintest['issue_date'] - X_traintest['issue_date'].min())  / np.timedelta64(1,'D')
X_internet['issue_date_delta'] = (X_internet['issue_date'] - X_internet['issue_date'].min())  / np.timedelta64(1,'D')

In [10]:
X_traintest['earlies_credit_mon'] = pd.to_datetime(X_traintest['earlies_credit_mon'])
X_internet['earlies_credit_mon'] = pd.to_datetime(X_internet['earlies_credit_mon'])

X_traintest['earlies_credit_mon_delta'] = (X_traintest['earlies_credit_mon'] - X_traintest['earlies_credit_mon'].min())  / np.timedelta64(1,'D')
X_internet['earlies_credit_mon_delta'] = (X_internet['earlies_credit_mon'] - X_internet['earlies_credit_mon'].min())  / np.timedelta64(1,'D')

In [11]:
X_traintest['issue_earlies_diff'] = (X_traintest['issue_date'] - X_traintest['earlies_credit_mon'])  / np.timedelta64(1,'D')
X_internet['issue_earlies_diff'] = (X_internet['issue_date'] - X_internet['earlies_credit_mon'])  / np.timedelta64(1,'D')

In [12]:
col_to_drop = ['issue_date', 'earlies_credit_mon']
X_traintest = X_traintest.drop(col_to_drop, axis=1)
X_internet = X_internet.drop(col_to_drop, axis=1)

#### employer_type, industry

#### 交叉特征

In [13]:
X_traintest['pro'] = X_traintest['interest'].mul(X_traintest['year_of_loan'])
X_traintest['loan_year'] = X_traintest['total_loan']/X_traintest['year_of_loan']
X_traintest['early_ratio'] = X_traintest['early_return_amount']/X_traintest['total_loan']
X_traintest['early_times_ratio'] = X_traintest['early_return']/X_traintest['year_of_loan']
X_traintest['recircle_ratio'] = X_traintest['recircle_b']/X_traintest['total_loan']
X_traintest['recircle_amt'] = X_traintest['recircle_b'].mul(X_traintest['total_loan'])

In [14]:
X_internet['pro'] = X_internet['interest'].mul(X_internet['year_of_loan'])
X_internet['loan_year'] = X_internet['total_loan']/X_internet['year_of_loan']
X_internet['early_ratio'] = X_internet['early_return_amount']/X_internet['total_loan']
X_internet['early_times_ratio'] = X_internet['early_return']/X_internet['year_of_loan']
X_internet['recircle_ratio'] = X_internet['recircle_b']/X_internet['total_loan']
X_internet['recircle_amt'] = X_internet['recircle_b'].mul(X_internet['total_loan'])

In [15]:
numerical_fea = list(X_traintest.select_dtypes(exclude=['object']).columns) 
category_fea = list(filter(lambda x: x not in numerical_fea,list(X_traintest.columns)))
# print("numerical_fea:",numerical_fea)
# print("category_fea:",category_fea)

### 模型 

In [16]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import catboost

In [17]:
all_fea = X_traintest.columns

In [18]:
X_train = X_traintest[X_traintest.is_train == 1][all_fea]
y_train = train_data['isDefault']

X_test = X_traintest[X_traintest.is_train == 0][all_fea]


In [19]:
X_train[category_fea] = X_train[category_fea].astype(str)
X_test[category_fea] = X_test[category_fea].astype(str)

X_train[numerical_fea] = X_train[numerical_fea].astype(float)
X_test[numerical_fea] = X_test[numerical_fea].astype(float)

In [20]:
model = catboost.CatBoostClassifier(cat_features = category_fea, random_state = 2080)
model.fit(X_train, y_train, verbose=400)

Learning rate set to 0.032658
0:	learn: 0.6488125	total: 159ms	remaining: 2m 38s
400:	learn: 0.1995525	total: 2.15s	remaining: 3.22s
800:	learn: 0.1618644	total: 4.23s	remaining: 1.05s
999:	learn: 0.1464660	total: 5.18s	remaining: 0us


In [21]:
y_pred = model.predict_proba(X_test)

### 模型改进

In [22]:
feature_importances = model.get_feature_importance(prettified=True,
                                                   thread_count=-1,
                                                   verbose=False)

In [23]:
tr_cols = set(X_traintest.columns)
same_col = list(tr_cols.intersection(set(X_internet.columns)))

tr_cols.difference(set(same_col))

{'app_type', 'is_train', 'known_dero', 'known_outstanding_loan'}

In [24]:
for i in range(feature_importances['Feature Id'].size):
    if (feature_importances['Feature Id'][i] in same_col):
        if feature_importances['Importances'][i]>1:
            X_traintest[feature_importances['Feature Id'][i]+'_cnt'] = category_encoders.count.CountEncoder().fit_transform(X_traintest[feature_importances['Feature Id'][i]])
            X_internet[feature_importances['Feature Id'][i]+'_cnt'] = category_encoders.count.CountEncoder().fit_transform(X_internet[feature_importances['Feature Id'][i]])


In [25]:
all_fea = X_traintest.columns
X_train = X_traintest[X_traintest.is_train == 1][all_fea]
X_train = pd.concat([X_train, train_data['isDefault']], axis=1)
y_train = train_data['isDefault']

X_test = X_traintest[X_traintest.is_train == 0][all_fea]

X_train = X_train.drop('is_train', axis=1)
X_test = X_test.drop('is_train', axis=1)

In [26]:
tr_cols = set(X_train.columns)
same_col = list(tr_cols.intersection(set(X_internet.columns)))
train_inteSame = X_internet[same_col].copy()

Inte_add_cos = list(tr_cols.difference(set(same_col)))
for col in Inte_add_cos:
    train_inteSame[col] = np.nan


In [27]:
numerical_fea = list(X_train.select_dtypes(exclude=['object']).columns) 
category_fea = list(filter(lambda x: x not in numerical_fea,list(X_train.columns)))


X_train[category_fea] = X_train[category_fea].astype(str)
train_inteSame[category_fea] = train_inteSame[category_fea].astype(str)

X_train[numerical_fea] = X_train[numerical_fea].astype(float)
train_inteSame[numerical_fea] = train_inteSame[numerical_fea].astype(float)

# Catboost

In [30]:
def train_model(data_, test_, y_, folds_):
    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in data_.columns if f not in ['loan_id', 'user_id', 'isDefault'] ]
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_)):
        trn_x, trn_y = data_[feats].iloc[trn_idx], y_.iloc[trn_idx]
        val_x, val_y = data_[feats].iloc[val_idx], y_.iloc[val_idx]
        clf = catboost.CatBoostClassifier(cat_features = category_fea, random_state = 2080)
        
        clf.fit(trn_x, trn_y, 
                eval_set= [(trn_x, trn_y), (val_x, val_y)], 
                verbose=400 #30
               )

        oof_preds[val_idx] = clf.predict_proba(val_x)[:, 1]
        sub_preds += clf.predict_proba(test_[feats])[:, 1] / folds_.n_splits
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))
        del clf, trn_x, trn_y, val_x, val_y
        gc.collect()
        
    print('Full AUC score %.6f' % roc_auc_score(y_, oof_preds)) 
    
    test_['isDefault'] = sub_preds

    return oof_preds, test_[['loan_id', 'isDefault']], feature_importance_df

In [31]:
folds = KFold(n_splits=5, shuffle=True, random_state=546789)
oof_preds, IntePre, importances = train_model(X_train, train_inteSame, y_train, folds)

Learning rate set to 0.058453
0:	learn: 0.6154447	test: 0.6155165	test1: 0.6161288	best: 0.6161288 (0)	total: 6.93ms	remaining: 6.92s
400:	learn: 0.1567822	test: 0.1568077	test1: 0.2389506	best: 0.2364307 (204)	total: 2.63s	remaining: 3.92s
800:	learn: 0.1019328	test: 0.1019579	test1: 0.2470964	best: 0.2364307 (204)	total: 5.25s	remaining: 1.3s
999:	learn: 0.0840897	test: 0.0841149	test1: 0.2517052	best: 0.2364307 (204)	total: 6.58s	remaining: 0us

bestTest = 0.2364306873
bestIteration = 204

Shrink model to first 205 iterations.
Fold  1 AUC : 0.886599
Learning rate set to 0.058453
0:	learn: 0.6154409	test: 0.6155131	test1: 0.6150933	best: 0.6150933 (0)	total: 7.44ms	remaining: 7.43s
400:	learn: 0.1561644	test: 0.1561906	test1: 0.2444803	best: 0.2385957 (156)	total: 2.8s	remaining: 4.18s
800:	learn: 0.1031193	test: 0.1031451	test1: 0.2576204	best: 0.2385957 (156)	total: 5.39s	remaining: 1.34s
999:	learn: 0.0856035	test: 0.0856285	test1: 0.2647247	best: 0.2385957 (156)	total: 6.81s	rema

In [32]:
IntePre['isDef'] = train_internet['is_default']
from sklearn.metrics import roc_auc_score
roc_auc_score(IntePre['isDef'],IntePre.isDefault)

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0.7329752753787245

In [33]:
InteId = IntePre.loc[IntePre.isDefault<0.02, 'loan_id'].tolist()

X_train['dataSourse'] = 1
X_test['dataSourse'] = 1
train_inteSame['dataSourse'] = 0
train_inteSame['isDefault'] = X_internet['is_default']
use_te = train_inteSame[train_inteSame.loan_id.isin( InteId )].copy()
data = pd.concat([ X_train,X_test,use_te]).reset_index(drop=True)


In [34]:
data[category_fea] = data[category_fea].astype(str)

data[numerical_fea] = data[numerical_fea].astype(float)

In [35]:
train = data[data['isDefault'].notna()]
test  = data[data['isDefault'].isna()]




y = train['isDefault']
folds = KFold(n_splits=5, shuffle=True, random_state=1997)
oof_preds, test_preds, importances = train_model(train, test, y, folds)

Learning rate set to 0.086034
0:	learn: 0.4326142	test: 0.4326310	test1: 0.4311381	best: 0.4311381 (0)	total: 14.5ms	remaining: 14.5s
400:	learn: 0.0281334	test: 0.0281398	test1: 0.0540453	best: 0.0516940 (74)	total: 4.26s	remaining: 6.36s
800:	learn: 0.0171809	test: 0.0171872	test1: 0.0579321	best: 0.0516940 (74)	total: 8.44s	remaining: 2.1s
999:	learn: 0.0136140	test: 0.0136201	test1: 0.0594003	best: 0.0516940 (74)	total: 10.5s	remaining: 0us

bestTest = 0.05169395486
bestIteration = 74

Shrink model to first 75 iterations.
Fold  1 AUC : 0.974899
Learning rate set to 0.086034
0:	learn: 0.4321620	test: 0.4321805	test1: 0.4323152	best: 0.4323152 (0)	total: 12.7ms	remaining: 12.7s
400:	learn: 0.0286567	test: 0.0286630	test1: 0.0566710	best: 0.0543771 (151)	total: 4.09s	remaining: 6.11s
800:	learn: 0.0179266	test: 0.0179328	test1: 0.0610709	best: 0.0543771 (151)	total: 8.31s	remaining: 2.06s
999:	learn: 0.0143418	test: 0.0143479	test1: 0.0637328	best: 0.0543771 (151)	total: 10.3s	remaini

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### 输出 

In [36]:
test_preds.rename({'loan_id': 'id'}, axis=1)[['id', 'isDefault']].to_csv('./submit/test_new_cb.csv', index=False)

In [37]:
test_preds


,loan_id,isDefault
14908,1000575.0,0.008986
14909,1028125.0,0.036979
14910,1010694.0,0.003642
14911,1026712.0,0.002939
14912,1002895.0,0.012328
...,...,...
19903,1008856.0,0.270302
19904,1016651.0,0.006606
19905,1024140.0,0.002429
19906,1014316.0,0.009418
